In [ ]:
!pip install pdfplumber
!pip install camelot-py[cv]
!pip install --upgrade pymupdf
!pip install google-generativeai
!pip install faiss-cpu
!pip install transformers tqdm pandas pytesseract pillow easyocr langchain langchain-community langchain_openai faiss-cpu rank_bm25 pdf2image
!pip install sentence_transformers

In [3]:
from dotenv import load_dotenv
import re
import os, glob
import pdfplumber
import camelot
import pymupdf
import numpy as np
import pandas as pd
from pathlib import Path
import time
import faiss, json
import collections
import fitz
import io
from PIL import Image
import pytesseract
from langchain_core.documents import Document
from openai import OpenAI
from TableRetrieval.table_ingestion import stage1_extract_and_save
from TableRetrieval.table_ingestion import store_in_faiss, save_metadata_mapping
from TableRetrieval.table_agentic_rag import TableAgenticRAG

load_dotenv()

True

# Agent CFO — Performance Optimization & Design

---
This is the starter notebook for your project. Follow the required structure below.


You will design and optimize an Agent CFO assistant for a listed company. The assistant should answer finance/operations questions using RAG (Retrieval-Augmented Generation) + agentic reasoning, with response time (latency) as the primary metric.

Your system must:
*   Ingest the company’s public filings.
*   Retrieve relevant passages efficiently.
*   Compute ratios/trends via tool calls (calculator, table parsing).
*   Produce answers with valid citations to the correct page/table.


## 1. Config & Secrets

Fill in your API keys in secrets. **Do not hardcode keys** in cells.

In [ ]:
import os

# Example:
# os.environ['GEMINI_API_KEY'] = 'your-key-here'
# os.environ['OPENAI_API_KEY'] = 'your-key-here'

COMPANY_NAME = "Google"

CHUNK_SIZE = 500  # number of words per chunk 

def generate_test_log_path_name(base_path: str): 
    # create the directory if not exist 
    os.makedirs(base_path, exist_ok=True) 
    existing_files = [f for f in os.listdir(base_path) if f.startswith("test_") and f.endswith(".json")] 
    existing_indices = [int(f.split("_")[1].split(".")[0]) for f in existing_files if f.split("_")[1].split(".")[0].isdigit()] 
    next_index = max(existing_indices) + 1 if existing_indices else 1 

    return f"{base_path}/test_{next_index}.json"


## 2. Data Download (Dropbox)

*   Annual Reports: last 3–5 years.
*   Quarterly Results Packs & MD&A (Management Discussion & Analysis).
*   Investor Presentations and Press Releases.
*   These files must be submitted later as a deliverable in the Dropbox data pack.
*   Upload them under `/content/data/`.

Scope limit: each team will ingest minimally 15 PDF files total.


In [4]:
DATA_DIR = "00-data"

# Annual reports (10-Ks)
annual_files = glob.glob(f"{DATA_DIR}/annuals/*.pdf")

# # Quarterly reports (10-Qs)
quarterly_files = glob.glob(f"{DATA_DIR}/quarterlies/*.pdf")

# Presentations
presentation_files = glob.glob(f"{DATA_DIR}/presentations/*.pdf")

In [5]:
for folder in ["annuals", "quarterlies", "presentations"]:

    files = glob.glob(f"{DATA_DIR}/{folder}/*.pdf")
    print(f"{folder}: {len(files)} files")

annuals: 3 files
quarterlies: 11 files
presentations: 2 files


## 3. System Requirements

**Retrieval & RAG**
*   Use a vector index (e.g., FAISS, LlamaIndex) + a keyword filter (BM25/ElasticSearch).
*   Citations must include: report name, year, page number, section/table.

**Agentic Reasoning**
*   Support at least 3 tool types: calculator, table extraction, multi-document compare.
*   Reasoning must follow a plan-then-act pattern (not a single unstructured call).

**Instrumentation**
*   Log timings for: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total.
*   Log: tokens used, cache hits, tools invoked.
*   Record p50/p95 latencies.

### Embeddings

In [ ]:
def create_embeddings(chunks, model="text-embedding-3-small"):
    """
    Create embeddings for chunks using OpenAI.
    Returns: chunks with 'embedding' field added
    """
    print(f"Creating embeddings with {model}...\n")
    print(f"   Total chunks: {len(chunks)}")
    
    client = OpenAI()

    # Split into batches of 50
    batch_size = 50
    total_batches = (len(chunks) + batch_size - 1) // batch_size

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        batch_num = i // batch_size + 1
        print(f"   Processing batch {batch_num} of {total_batches}...")

        contents = [chunk['content'] for chunk in batch]
        
        # Embed batch
        response = client.embeddings.create(
            model=model,
            input=contents
        )
        
        # Add embeddings to chunks
        for j, chunk in enumerate(batch):
            chunk['embedding'] = response.data[j].embedding

        # Small delay to respect rate limits
        if i + batch_size < len(chunks):
                time.sleep(0.6) # Adjust as needed
    
    print(f"Created {len(chunks)} embeddings\n")
    return chunks

### Table Ingestion

In [ ]:
def stage2_create_embeddings(json_file):
    """
    Load extracted tables from JSON and create embeddings.
    This is where you spend OpenAI tokens.
    """
    load_dotenv()
    
    DATA_DIR = "00-data"
    
    print("="*80)
    print("STAGE 2: CREATING EMBEDDINGS")
    print("="*80)
    print()
    
    # Load the extracted tables
    print(f"Loading: {json_file}")
    with open(json_file, 'r') as f:
        table_chunks = json.load(f)
    
    print(f"Loaded {len(table_chunks)} tables")
    
    print("\nCreating embeddings...")
    embedded_chunks = create_embeddings(table_chunks)
    
    print("\nStoring in FAISS...")
    faiss_index = store_in_faiss(
        embedded_chunks,
        faiss_index_path=f"{DATA_DIR}/base/faiss_table_index"
    )
    
    print("\nSaving metadata...")
    save_metadata_mapping(
        embedded_chunks,
        mapping_path=f"{DATA_DIR}/base/faiss_table_metadata.json"
    )
    
    print(f"\n{'='*80}")
    print("COMPLETE!")
    print("="*80)
    print(f"FAISS index: {DATA_DIR}/base/faiss_table_index")
    print(f"Metadata: {DATA_DIR}/base/faiss_table_metadata.json")
    print()

In [ ]:
# Insert table ingestion code here
stage1_extract_and_save()

In [ ]:
stage2_create_embeddings(f"{DATA_DIR}/extracted_tables.json")

## Text Ingestion

In [1]:
from TextRetrieval.TextExtractor import extract_text_from_pdf

extract_text_from_pdf(); 

No PDF paths provided, loading from default annual and quarterly files.
Processing 14 PDFs from all folders
PDF paths: ['00-data/annuals\\goog-10-k-2022.pdf', '00-data/annuals\\goog-10-k-2023-final.pdf', '00-data/annuals\\goog-10-k-2024.pdf'] ...

=== Processing: goog-10-k-2022.pdf ===
Page 1 → Text length: 2689
Page 2 → Text length: 3256
Page 3 → Text length: 1411
Page 4 → Text length: 3678
Page 5 → Text length: 4775
Page 6 → Text length: 4730
Page 7 → Text length: 4785
Page 8 → Text length: 4067
Page 9 → Text length: 5062
Page 10 → Text length: 5072
Page 11 → Text length: 5701
Page 12 → Text length: 5800
Page 13 → Text length: 5466
Page 14 → Text length: 5263
Page 15 → Text length: 5538
Page 16 → Text length: 6122
Page 17 → Text length: 6097
Page 18 → Text length: 5392
Page 19 → Text length: 5574
Page 20 → Text length: 5826
Page 21 → Text length: 5512
Page 22 → Text length: 5065
Page 23 → Text length: 4494
Page 24 → Text length: 3456
Page 25 → Text length: 1205
Page 26 → Text length:

### Slides Ingestion

In [ ]:
def extract_slides_fitz(pdf_path, output_dir, lower_crop_extra=200):
    os.makedirs(output_dir, exist_ok=True)
    pdf = fitz.open(pdf_path)
    print(f"[INFO] Loaded '{pdf_path}' with {len(pdf)} pages.")

    for i, page in enumerate(pdf, start=1):
        pix = page.get_pixmap(dpi=200)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # Crop lower for slides 1–2
        if i in [1, 2]:
            w, h = img.size
            crop_box = (0, 0, w, min(h + lower_crop_extra, h))
            img = img.crop(crop_box)

        img.save(os.path.join(output_dir, f"slide_{i:02d}.png"), "PNG")

    print(f"Extracted {len(pdf)} slides from {pdf_path}")

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
def ocr_folder(folder, label):
    docs = []
    for fname in sorted(os.listdir(folder)):
        if fname.endswith(".png"):
            path = os.path.join(folder, fname)
            text = pytesseract.image_to_string(Image.open(path))
            if text.strip():
                docs.append(Document(
                    page_content=text,
                    metadata={"image_path": path, "source_report": label}
                ))
    return docs

In [ ]:
# Extract slides and OCR
pdf_path = []
for folder in ["presentations"]:
    files = glob.glob(f"{DATA_DIR}/{folder}/*.pdf")
    pdf_path.extend(files)

print(f"Processing {len(pdf_path)} PDFs from all folders")

docs = []
for pdf in pdf_path:
    # Create output folder based on PDF name
    pdf_name = os.path.splitext(os.path.basename(pdf))[0]
    slide_folder = f"{DATA_DIR}/presentations/slides_{pdf_name}"
    
    print(f"Extracting slides from {pdf_name}...")
    extract_slides_fitz(pdf, slide_folder)
    
    # OCR the extracted slides
    pdf_docs = ocr_folder(slide_folder, pdf_name)
    docs.extend(pdf_docs)

print(f"Loaded {len(docs)} slide documents from {len(pdf_path)} PDFs.")

## TEXT FAISS BUILDER
### CREATE THE CHUNKS 
### BUILD THE INDICES BASE OFF THE CHUNKS

In [1]:
from TextRetrieval.TextFaissBuilder import create_chunks, built_indices

chunks = create_chunks();
built_indices(chunks); 

Created 1223 chunks from the documents.
Building index for section: cover_page with 46 chunks.
Embeddings shape for section cover_page: (46, 1536)
Building index for section: equity with 192 chunks.
Embeddings shape for section equity: (192, 1536)
Building index for section: risk_factors with 117 chunks.
Embeddings shape for section risk_factors: (117, 1536)
Building index for section: mdna with 88 chunks.
Embeddings shape for section mdna: (88, 1536)
Building index for section: summary_financial_data with 192 chunks.
Embeddings shape for section summary_financial_data: (192, 1536)
Building index for section: legal_proceedings with 122 chunks.
Embeddings shape for section legal_proceedings: (122, 1536)
Building index for section: cash_flow with 65 chunks.
Embeddings shape for section cash_flow: (65, 1536)
Building index for section: financial_statements with 187 chunks.
Embeddings shape for section financial_statements: (187, 1536)
Building index for section: market_risk_disclosures wi

## 4. Baseline Pipeline

**Baseline (starting point)**
*   Naive chunking.
*   Single-pass vector search.
*   One LLM call, no caching.

In [ ]:
# TODO: Implement baseline retrieval + generation


## 5. Benchmark Runner

Run these 3 standardized queries. Produce JSON then prose answers with citations. These are the standardized queries.

*   Gross Margin Trend (or NIM if Bank)
    *   Query: "Report the Gross Margin (or Net Interest Margin, if a bank) over the last 5 quarters, with values."
    *   Expected Output: A quarterly table of Gross Margin % (or NIM % if bank).

*   Operating Expenses (Opex) YoY for 3 Years
    *   Query: "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."
    *   Expected Output: A 3-year Opex table (absolute numbers and % change).

*   Operating Efficiency Ratio
    *   Query: "Calculate the Operating Efficiency Ratio (Opex ÷ Operating Income) for the last 3 fiscal years, showing the working."
    *   Expected Output: Table with Opex, Operating Income, and calculated ratio for 3 years.

In [ ]:
# TODO: Implement benchmark runner


## 6. Instrumentation

Log timings: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total. Log tokens, cache hits, tools.

In [ ]:
# Example instrumentation schema
import pandas as pd
logs = pd.DataFrame(columns=['Query','T_ingest','T_retrieve','T_rerank','T_reason','T_generate','T_total','Tokens','CacheHits','Tools'])
logs

## 7. Optimizations

**Required Optimizations**

Each team must implement at least:
*   2 retrieval optimizations (e.g., hybrid BM25+vector, smaller embeddings, dynamic k).
*   1 caching optimization (query cache or ratio cache).
*   1 agentic optimization (plan pruning, parallel sub-queries).
*   1 system optimization (async I/O, batch embedding, memory-mapped vectors).

In [ ]:
# TODO: Implement optimizations


### Table Agentic Optimization

In [6]:
FAISS_INDEX = f"{DATA_DIR}/base/faiss_table_index"
METADATA_JSON = f"{DATA_DIR}/base/faiss_table_metadata.json"

# Create the agent
table_agent = TableAgenticRAG(
    faiss_index_path=FAISS_INDEX,
    metadata_json_path=METADATA_JSON
)

query = "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."


print("\nRunning agent query...\n")
result = table_agent.query(query, verbose=True)


print("\n====================== ANSWER ======================")
print(result["answer"])
print("===================================================\n")


print("Sources:", result["sources"])


Running agent query...


QUERY: Show Operating Expenses for the last 3 fiscal years, year-on-year comparison.

Cache MISS. Proceeding with retrieval...

PLANNING PHASE
------------------------------------------------------------
Query Type: comparison
Metrics Needed: Operating Expenses 2023, Operating Expenses 2024, Operating Expenses 2025
Planned Searches: 3
  1. [tables] Operating Expenses 2023
     Reason: To retrieve structured financial data for accurate year-on-year comparison.
  2. [tables] Operating Expenses 2024
     Reason: To retrieve structured financial data for accurate year-on-year comparison.
  3. [tables] Operating Expenses 2025
     Reason: To retrieve structured financial data for accurate year-on-year comparison.

RETRIEVAL PHASE
------------------------------------------------------------

Total chunks retrieved: 15
Unique chunks: 9

GENERATE ANSWER PHASE
------------------------------------------------------------
Answer generated: 1760 characters

Cache storage 

In [7]:
query2 = "What is the operating expense for the last 3 fiscal years, year-on-year comparison."
print("\nRunning agent query 2...\n")
result = table_agent.query(query2, verbose=True)

print("\n====================== ANSWER ======================")
print(result["answer"])
print("===================================================\n")


print("Sources:", result["sources"])


Running agent query 2...


QUERY: What is the operating expense for the last 3 fiscal years, year-on-year comparison.

Cache HIT:
  Similarity: 0.9032
  Cached Query: Show Operating Expenses for the last 3 fiscal years, year-on-year comparison.


====================== ANSWER ======================
To show the Operating Expenses for the last three fiscal years and provide a year-on-year comparison, we will focus on the General and Administrative (G&A) expenses as a proxy for operating expenses, as this is the data available. Here is the data extracted from the provided documents:

### General and Administrative Expenses (in millions)

| Year | G&A Expenses | Source                  |
|------|--------------|-------------------------|
| 2021 | $13,510      | (Source: goog-10-k-2022)|
| 2022 | $15,724      | (Source: goog-10-k-2023)|
| 2023 | $16,425      | (Source: goog-10-k-2024)|

### Year-on-Year Comparison

1. **2021 to 2022:**
   - **Calculation:** 
     \[
     \text{Percentage In

### TEXT Agentic Optimization

In [2]:
from TextRetrieval.Agent import text_agent_executor

query = "" \
"What is the operating expense for the last 3 fiscal years, year-on-year comparison."
text_agent_executor(query)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find the operating expenses for the last 3 fiscal years from the financial filings. I will first check which sections are available to ensure I query the right section for operating expenses.

Action: available_sections_retriever
Action Input: ""[INFO] FAISS indices already initialized, reusing existing indices.

Observation: ['balance_sheet', 'cash_flow', 'controls_procedures', 'cover_page', 'equity', 'exhibits', 'financial_statements', 'income_statement', 'legal_proceedings', 'market_risk_disclosures', 'mdna', 'risk_factors', 'segment_info', 'signatures', 'summary_financial_data']
Thought:Thought: The operating expenses are typically found in the income statement section. I will retrieve the operating expenses for the last 3 fiscal years from the income_statement section.

Action: retriever
Action Input: '{"query": "operating expenses for the last 3 fiscal years", "sections": ["income_statement"]}'[IN

## 8. Results & Plots

Show baseline vs optimized. Include latency plots (p50/p95) and accuracy tables.

In [ ]:
# TODO: Generate plots with matplotlib
